In [2]:
import pandas as pd

In [7]:
data = pd.read_json('counterfact.json', orient='records').set_index('case_id')
data

,pararel_idx,requested_rewrite,paraphrase_prompts,neighborhood_prompts,attribute_prompts,generation_prompts
case_id,,,,,,
0,2796,"{'prompt': 'The mother tongue of {} is', 'rela...",[Shayna does this and Yossel goes still and di...,"[The mother tongue of Léon Blum is, The native...","[J. R. R. Tolkien is a native speaker of, The ...","[Danielle Darrieux's mother tongue is, Where D..."
1,19501,"{'prompt': 'The official religion of {} is', '...",[The school chiefly served tribal girls of Dan...,"[The official religion of Charles Aznavour is,...",[Dave Chappelle is affiliated with the religio...,[Edwin of Northumbria's religious values stron...
2,13704,"{'prompt': '{}, the', 'relation_id': 'P1303', ...",[Initially and are zero and is false. Toko ...,"[Paul McCartney plays the instrument, John Len...","[Robert Radecke performs on the, Peter Igelhof...",[Toko Yasuda produces the most amazing music o...
3,6791,"{'prompt': '{}, which is located in', 'relatio...",[and Sallie Beavers Riley. Autonomous Universi...,"[Biure is located in, Ripollès, located in, Eb...","[SKF is located in, Köping Municipality, in, U...",[One can get to Autonomous University of Madri...
4,14712,{'prompt': 'What is the twin city of {}? It is...,"[Overall, however, Nápravník stayed true to Pu...","[What is the twin city of Los Angeles? It is, ...","[Sacramento is a twin city of, The twin city o...","[Lyon's twin city is known for, Lyon's twin ci..."
...,...,...,...,...,...,...
21914,3468,"{'prompt': '{}, speaker of', 'relation_id': 'P...","[Presently, he is Chief Justice of Meghalaya H...","[The mother tongue of Jean-Luc Picard is, Loui...","[The native language of El Lissitzky is, Mikha...","[Where Georges Bernier is from, people speak t..."
21915,18312,"{'prompt': 'The language used by {} is', 'rela...",[The federal courts ruled otherwise. Jean-Pier...,"[Georges Pompidou speaks, George Sand speaks t...","[The language used by Grey Griffin is, Alberto...","[Jean-Pierre Dionnet was born in, Jean-Pierre ..."
21916,7584,{'prompt': 'Which position does {} play? They ...,"[Also felt in Mendoza and San Juan Provinces, ...","[Bruce Chen, who plays the position, Motoshi F...","[Al Silvera plays in the position of, John Rod...",[The expertise of Bong Jung-keun becomes impor...


In [10]:
sample = data.loc[:500].reset_index(drop=True)

In [12]:
new_dataset = pd.DataFrame(
    columns=['text', 'instruction', 'good_response'],
    index=sample.index
)

In [23]:
new_dataset.loc[:, 'text'] = sample.apply(lambda z: z['requested_rewrite']['prompt'].format(z['requested_rewrite']['subject']) + ' ' + z['requested_rewrite']['target_true']['str'], axis=1)
new_dataset

,text,instruction,good_response
0,The mother tongue of Danielle Darrieux is French,NaN,NaN
1,The official religion of Edwin of Northumbria ...,NaN,NaN
2,"Toko Yasuda, the guitar",NaN,NaN
3,"Autonomous University of Madrid, which is loca...",NaN,NaN
4,What is the twin city of Lyon? It is Beirut,NaN,NaN
...,...,...,...
496,Legg Mason originated in Baltimore,NaN,NaN
497,"Fa Ngum, who is a citizen of Laos",NaN,NaN
498,"Thomas Erle, a citizen of England",NaN,NaN
499,Porto is a twin city of Bristol,NaN,NaN


In [25]:
new_dataset.to_json('new_dataset_temp.json', orient='records', indent=2)

At this step, I send the file to gpt4 and ask this prompt:

I want you to fill in the nulls in this file. for each record, convert the text field in the form of a question, and put it in the instruct field. Then, answer that question with some explanation and put it in the good_response field. respond with a json format with the requested fields filled.

This should be automated with an API later. with one row in each API call

In [4]:
new_dataset_gpt4 = pd.read_json('new_dataset_gpt-4.json', orient='records')
new_dataset_gpt4

,text,instruction,good_response
0,The mother tongue of Danielle Darrieux is French,What is the mother tongue of Danielle Darrieux?,The mother tongue of Danielle Darrieux is Fren...
1,The official religion of Edwin of Northumbria ...,What is the official religion of Edwin of Nort...,The official religion of Edwin of Northumbria ...
2,"Toko Yasuda, the guitar",What instrument does Toko Yasuda play?,"Toko Yasuda plays the guitar, which is a strin..."
3,"Autonomous University of Madrid, which is loca...",Where is the Autonomous University of Madrid l...,The Autonomous University of Madrid is located...
4,What is the twin city of Lyon? It is Beirut,What is the twin city of Lyon?,The twin city of Lyon is Beirut. Twin cities o...
...,...,...,...
143,The original language of Delusions of Grandeur...,What is the original language of Delusions of ...,The original language of Delusions of Grandeur...
144,The law in Gibraltar declares the language Eng...,What language does the law in Gibraltar declare?,The law in Gibraltar declares the language Eng...
145,Falling Down was from France,Where was Falling Down from?,Falling Down was from France. It is a 1993 thr...
146,2010 Winter Paralympics can be found in Vancouver,Where can the 2010 Winter Paralympics be found?,The 2010 Winter Paralympics can be found in Va...
